In [12]:
import torch
import torchvision
import torchvision.transforms as transforms
import pandas as pd
from torchvision.utils import save_image
import torch
import torchvision
from openTSNE import TSNE
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import os

In [42]:

# --------------------------------------
# PARAMETERS
# --------------------------------------

class Parameters:
    NUMBER_OF_SAMPLES_LIST = [700, 3000, 12000]
    PERPLEXITIES = [10, 30, 50, 80, 100]
    DATASET = ['imagenette', 'imagewoof']
    DIMENSIONALITY_REDUCTION_METHODS = ['tsne', 'pca']
    LAYER_IDX_LIST = []
    ACTIVATIONS = []
    ANNOTATIONS_IMAGENETTE = 'datasets/imagenette2-320/noisy_imagenette.csv'
    ANNOTATIONS_IMAGEWOOF = 'datasets/imagewoof2-320/noisy_imagewoof.csv'
    DIR_IMAGENETTE = 'datasets/imagenette2-320'
    DIR_IMAGEWOOF = 'datasets/imagewoof2-320'
    BATCH_SIZE = 32
    NUMBER_OF_SAMPLES = 10



In [33]:
# --------------------------------------
# Process mapping sysnsetid (label) to category
# --------------------------------------

mapping_label_to_category = {}
mapping_category_to_label = {}

with open('datasets/sysnet_to_category.txt')as f:
    for line in f:
        line_items = line.split(':')
        label = line_items[0]
        #print(line_items)
        category = line_items[1].split(',')[0][1:].strip()
        
        mapping_label_to_category[label] = category
        mapping_category_to_label[category] = label
    

In [34]:
from torchvision.models import vgg16, VGG16_Weights

#Imagenet weights
weights = VGG16_Weights.DEFAULT

categories = weights.meta['categories']

mapping_label_id_to_category = {}
mapping_label_category_to_id = {}

for idx, category  in enumerate(categories):
    mapping_label_id_to_category[idx] = category
    mapping_label_category_to_id[category] = idx


#Create dataframe out of it

In [38]:
# --------------------------------------
# Create mapping dataframe
# --------------------------------------


labels = []
label_ids = []
categories = []


for label, category in mapping_label_to_category.items():
    labels.append(label)
    categories.append(category)
    label_ids.append(mapping_label_category_to_id[category])


data = {
    'label':  labels,
    'label_id': label_ids,
    'category': categories,
}

mapping_data_points_df = pd.DataFrame(data) 
mapping_data_points_df

,label,label_id,category
0,n02119789,278,kit fox
1,n02100735,212,English setter
2,n02096294,193,Australian terrier
3,n02066245,147,grey whale
4,n02509815,387,lesser panda
...,...,...,...
995,n02086646,156,Blenheim spaniel
996,n02091244,173,Ibizan hound
997,n02089973,167,English foxhound
998,n02105251,226,briard


In [ ]:
#Solving array via numpy

test = np.arange(10)
np.savetxt('array_2d.csv', test, delimiter=',')
loaded_array_2d = np.load('array_2d.npy')

In [43]:
data = pd.read_csv(Parameters.ANNOTATIONS_IMAGENETTE)
data

,path,noisy_labels_0,noisy_labels_1,noisy_labels_5,noisy_labels_25,noisy_labels_50,is_valid
0,train/n02979186/n02979186_9036.JPEG,n02979186,n02979186,n02979186,n02979186,n02979186,False
1,train/n02979186/n02979186_11957.JPEG,n02979186,n02979186,n02979186,n02979186,n03000684,False
2,train/n02979186/n02979186_9715.JPEG,n02979186,n02979186,n02979186,n03417042,n03000684,False
3,train/n02979186/n02979186_21736.JPEG,n02979186,n02979186,n02979186,n02979186,n03417042,False
4,train/n02979186/ILSVRC2012_val_00046953.JPEG,n02979186,n02979186,n02979186,n02979186,n03394916,False
...,...,...,...,...,...,...,...
13389,val/n03425413/n03425413_17521.JPEG,n03425413,n03425413,n03425413,n03425413,n03425413,True
13390,val/n03425413/n03425413_20711.JPEG,n03425413,n03425413,n03425413,n03425413,n03425413,True
13391,val/n03425413/n03425413_19050.JPEG,n03425413,n03425413,n03425413,n03425413,n03425413,True
13392,val/n03425413/n03425413_13831.JPEG,n03425413,n03425413,n03425413,n03425413,n03425413,True


In [17]:
data['label'] = data.iloc[0]['path'].split('/')[1]



In [15]:
img_path = os.path.join(Parameters.DIR_IMAGENETTE, data.iloc[0]['path'])
img_path

'datasets/imagenette2-320/train/n02979186/n02979186_9036.JPEG'

In [19]:
data['label'] = data.iloc[0]['path'].split('/')[1]


,path,noisy_labels_0,noisy_labels_1,noisy_labels_5,noisy_labels_25,noisy_labels_50,is_valid,label
0,train/n02979186/n02979186_9036.JPEG,n02979186,n02979186,n02979186,n02979186,n02979186,False,n02979186
1,train/n02979186/n02979186_11957.JPEG,n02979186,n02979186,n02979186,n02979186,n03000684,False,n02979186
2,train/n02979186/n02979186_9715.JPEG,n02979186,n02979186,n02979186,n03417042,n03000684,False,n02979186
3,train/n02979186/n02979186_21736.JPEG,n02979186,n02979186,n02979186,n02979186,n03417042,False,n02979186
4,train/n02979186/ILSVRC2012_val_00046953.JPEG,n02979186,n02979186,n02979186,n02979186,n03394916,False,n02979186
...,...,...,...,...,...,...,...,...
13389,val/n03425413/n03425413_17521.JPEG,n03425413,n03425413,n03425413,n03425413,n03425413,True,n02979186
13390,val/n03425413/n03425413_20711.JPEG,n03425413,n03425413,n03425413,n03425413,n03425413,True,n02979186
13391,val/n03425413/n03425413_19050.JPEG,n03425413,n03425413,n03425413,n03425413,n03425413,True,n02979186
13392,val/n03425413/n03425413_13831.JPEG,n03425413,n03425413,n03425413,n03425413,n03425413,True,n02979186


In [49]:
mapping_label_to_category['n02979186']

'cassette player'

In [44]:
annotations_df = pd.read_csv(Parameters.ANNOTATIONS_IMAGENETTE)


In [63]:
annotations_df['label'] = annotations_df['path'].str.split('/').str[1]


In [65]:
df_merged = annotations_df.merge(mapping_data_points_df, left_on='label', right_on='label', how='left')

,path,noisy_labels_0,noisy_labels_1,noisy_labels_5,noisy_labels_25,noisy_labels_50,is_valid,label,label_id,category
0,train/n02979186/n02979186_9036.JPEG,n02979186,n02979186,n02979186,n02979186,n02979186,False,n02979186,482,cassette player
1,train/n02979186/n02979186_11957.JPEG,n02979186,n02979186,n02979186,n02979186,n03000684,False,n02979186,482,cassette player
2,train/n02979186/n02979186_9715.JPEG,n02979186,n02979186,n02979186,n03417042,n03000684,False,n02979186,482,cassette player
3,train/n02979186/n02979186_21736.JPEG,n02979186,n02979186,n02979186,n02979186,n03417042,False,n02979186,482,cassette player
4,train/n02979186/ILSVRC2012_val_00046953.JPEG,n02979186,n02979186,n02979186,n02979186,n03394916,False,n02979186,482,cassette player
...,...,...,...,...,...,...,...,...,...,...
13389,val/n03425413/n03425413_17521.JPEG,n03425413,n03425413,n03425413,n03425413,n03425413,True,n03425413,571,gas pump
13390,val/n03425413/n03425413_20711.JPEG,n03425413,n03425413,n03425413,n03425413,n03425413,True,n03425413,571,gas pump
13391,val/n03425413/n03425413_19050.JPEG,n03425413,n03425413,n03425413,n03425413,n03425413,True,n03425413,571,gas pump
13392,val/n03425413/n03425413_13831.JPEG,n03425413,n03425413,n03425413,n03425413,n03425413,True,n03425413,571,gas pump


In [71]:
def process_annotations(annotations_df):
    
    #clean up
    annotations_df = annotations_df.drop(columns=['noisy_labels_0', 'noisy_labels_1','noisy_labels_5','noisy_labels_25','noisy_labels_50','is_valid'])
    
    #parse true labels from dir
    annotations_df['label'] = annotations_df['path'].str.split('/').str[1]
    
    #include other mappings
    annotations_df = annotations_df.merge(mapping_data_points_df, left_on='label', right_on='label', how='left')

    #include id
    annotations_df['uid'] = range(0, len(annotations_df))
    
    return annotations_df
    

In [72]:
annotations_df = pd.read_csv(Parameters.ANNOTATIONS_IMAGENETTE)
annotations_df = process_annotations(annotations_df)
annotations_df


,path,label,label_id,category,uid
0,train/n02979186/n02979186_9036.JPEG,n02979186,482,cassette player,0
1,train/n02979186/n02979186_11957.JPEG,n02979186,482,cassette player,1
2,train/n02979186/n02979186_9715.JPEG,n02979186,482,cassette player,2
3,train/n02979186/n02979186_21736.JPEG,n02979186,482,cassette player,3
4,train/n02979186/ILSVRC2012_val_00046953.JPEG,n02979186,482,cassette player,4
...,...,...,...,...,...
13389,val/n03425413/n03425413_17521.JPEG,n03425413,571,gas pump,13389
13390,val/n03425413/n03425413_20711.JPEG,n03425413,571,gas pump,13390
13391,val/n03425413/n03425413_19050.JPEG,n03425413,571,gas pump,13391
13392,val/n03425413/n03425413_13831.JPEG,n03425413,571,gas pump,13392


In [52]:
annotations_df['label'] = annotations_df.iloc[0]['path'].split('/')[1]
annotations_df

,path,noisy_labels_0,noisy_labels_1,noisy_labels_5,noisy_labels_25,noisy_labels_50,is_valid,label
0,train/n02979186/n02979186_9036.JPEG,n02979186,n02979186,n02979186,n02979186,n02979186,False,n02979186
1,train/n02979186/n02979186_11957.JPEG,n02979186,n02979186,n02979186,n02979186,n03000684,False,n02979186
2,train/n02979186/n02979186_9715.JPEG,n02979186,n02979186,n02979186,n03417042,n03000684,False,n02979186
3,train/n02979186/n02979186_21736.JPEG,n02979186,n02979186,n02979186,n02979186,n03417042,False,n02979186
4,train/n02979186/ILSVRC2012_val_00046953.JPEG,n02979186,n02979186,n02979186,n02979186,n03394916,False,n02979186
...,...,...,...,...,...,...,...,...
13389,val/n03425413/n03425413_17521.JPEG,n03425413,n03425413,n03425413,n03425413,n03425413,True,n02979186
13390,val/n03425413/n03425413_20711.JPEG,n03425413,n03425413,n03425413,n03425413,n03425413,True,n02979186
13391,val/n03425413/n03425413_19050.JPEG,n03425413,n03425413,n03425413,n03425413,n03425413,True,n02979186
13392,val/n03425413/n03425413_13831.JPEG,n03425413,n03425413,n03425413,n03425413,n03425413,True,n02979186


In [53]:
mapping_data_points_df[mapping_data_points_df['label'] == annotations_df['label']]

ValueError: Can only compare identically-labeled Series objects

In [80]:
annotations_df.iloc[0]['path']

'train/n02979186/n02979186_9036.JPEG'

In [92]:

from torch.utils.data import Dataset
import os
import pandas as pd
from torchvision.io import read_image
from torchvision.transforms import InterpolationMode, ToPILImage
from PIL import Image
import numpy

# Define the transformation pipeline
transform = transforms.Compose([
    # Resize the image to 256x256 using bilinear interpolation
    transforms.Resize(256, interpolation=InterpolationMode.BILINEAR),
    
    # Central crop to 224x224
    transforms.CenterCrop(224),
    
    # Convert the image to a Tensor and rescale to [0.0, 1.0]
    transforms.ToTensor(),
    
    # Normalize the image with the given mean and std
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])



class CustomImageDataset(Dataset):
    def __init__(self, dataset_dir, annotations_df, transform=None, target_transform=None):
        self.annotations_df = annotations_df
        self.dataset_dir = dataset_dir
        
        
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.annotations_df)

    def __getitem__(self, idx):
        img_path = os.path.join(self.dataset_dir, self.annotations_df.iloc[idx]['path'])
        image_tensor = read_image(img_path)
        
        # Convert tensor to PIL Image before applying transformations
        #image = ToPILImage()(image_tensor)
        
        image = Image.open(img_path)
        
        label = self.annotations_df.iloc[idx]['label']
        label_id = self.annotations_df.iloc[idx]['label_id']
        category = self.annotations_df.iloc[idx]['category']
        uid = self.annotations_df.iloc[idx]['uid']
        
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
            
        return image, label, label_id, category, uid
    
    
annotations_imagenette_df = pd.read_csv(Parameters.ANNOTATIONS_IMAGENETTE)
annotations_imagenette_df = process_annotations(annotations_imagenette_df)
imagenette_data_set = CustomImageDataset(Parameters.DIR_IMAGENETTE, annotations_imagenette_df, transform=transform)
trainloader = torch.utils.data.DataLoader(imagenette_data_set, batch_size=Parameters.NUMBER_OF_SAMPLES, shuffle=True)

dataiter = iter(trainloader)
image, label, label_id, category, uid = next(dataiter)

print(image)
print(label)


RuntimeError: Numpy is not available

In [23]:
from torchvision.models import vgg16, VGG16_Weights

#Imagenet weights
weights = VGG16_Weights.DEFAULT

categories = weights.meta['categories']

mapping_label_id_to_category = {}

for idx, category  in enumerate(categories):
    mapping_label_id_to_category[idx] = category



{0: 'tench',
 1: 'goldfish',
 2: 'great white shark',
 3: 'tiger shark',
 4: 'hammerhead',
 5: 'electric ray',
 6: 'stingray',
 7: 'cock',
 8: 'hen',
 9: 'ostrich',
 10: 'brambling',
 11: 'goldfinch',
 12: 'house finch',
 13: 'junco',
 14: 'indigo bunting',
 15: 'robin',
 16: 'bulbul',
 17: 'jay',
 18: 'magpie',
 19: 'chickadee',
 20: 'water ouzel',
 21: 'kite',
 22: 'bald eagle',
 23: 'vulture',
 24: 'great grey owl',
 25: 'European fire salamander',
 26: 'common newt',
 27: 'eft',
 28: 'spotted salamander',
 29: 'axolotl',
 30: 'bullfrog',
 31: 'tree frog',
 32: 'tailed frog',
 33: 'loggerhead',
 34: 'leatherback turtle',
 35: 'mud turtle',
 36: 'terrapin',
 37: 'box turtle',
 38: 'banded gecko',
 39: 'common iguana',
 40: 'American chameleon',
 41: 'whiptail',
 42: 'agama',
 43: 'frilled lizard',
 44: 'alligator lizard',
 45: 'Gila monster',
 46: 'green lizard',
 47: 'African chameleon',
 48: 'Komodo dragon',
 49: 'African crocodile',
 50: 'American alligator',
 51: 'triceratops',
 

In [ ]:
# --------------------------------------
# Load data
# --------------------------------------

#transform? Todo


imagenette_data_set = CustomImageDataset(Parameters.DIR_IMAGENETTE, Parameters.ANNOTATIONS_IMAGENETTE)
imagewoof_data_set = CustomImageDataset(Parameters.DIR_IMAGEWOOF, Parameters.ANNOTATIONS_IMAGEWOOF)
trainloader = torch.utils.data.DataLoader(imagenette_data_set, batch_size=Parameters.NUMBER_OF_SAMPLES, shuffle=True)

dataiter = iter(trainloader)
images, labels = next(dataiter)
                
